In [44]:
import pandas as pd
import numpy as np

In [85]:
df = pd.read_csv(
    '/home/oalves/ombede/can.log',
    sep=' ',
    header=None,
    names=['frame', 'frameid', 'timestamp', *[f'b{i}' for i in range(8)]],
    on_bad_lines='skip'
)

print(df.head())

/tmp/ipykernel_1899505/1778662120.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


   frame frameid     timestamp  b0  b1  b2  b3  b4  b5  b6  b7
0  frame     430  1.766951e+09  86  00  AF  C6  64  00  12  90
1  frame     434  1.766951e+09  03  4E  04  00  0F  FF  00  00
2  frame     430  1.766951e+09  82  00  AF  C6  64  00  12  90
3  frame     434  1.766951e+09  03  4D  04  00  0F  FF  00  00
4  frame     43C  1.766951e+09  32  00  39  4A  16  D9  88  14


In [86]:
def parsehex(x):
    try:
        return int(str(x), 16)
    except:
        print(x)
        return np.nan

for col in ('frameid', *(f'b{i}' for i in range(8))):
    print(col)
    df[col].apply(parsehex)

frameid
b0
b1
b2
b3
b4
b5
b6
b7


In [87]:
diffs = df['timestamp'].diff()

new_group_flag = diffs > 5

# 4. Create a unique ID for each group using a cumulative sum
df['trip'] = new_group_flag.cumsum()
df['line_no'] = df.index
# 5. Aggregate the results
report = df.groupby('trip').agg(
    start_timestamp=('timestamp', 'min'),
    end_timestamp=('timestamp', 'max'),
    line_count=('timestamp', 'count'),
    start_line=('line_no', 'min'),
    end_line=('line_no', 'max')
).reset_index(drop=True)

report['duration'] = (report['end_timestamp'] - report['start_timestamp'])/60

print(report)


   start_timestamp  end_timestamp  line_count  start_line  end_line  \
0     1.766951e+09   1.766951e+09        3260           0      3259   
1     1.766951e+09   1.766954e+09      280827        3260    284086   
2     1.766954e+09   1.767005e+09    16551535      284087  16835621   

     duration  
0    2.582077  
1   45.575455  
2  857.721549  


In [66]:
df['timestamp'].is_monotonic_increasing

False